In [2]:
!pip install scipy

  Using cached scipy-1.15.2-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached scipy-1.15.2-cp311-cp311-win_amd64.whl (41.2 MB)


In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
import scipy.stats as stats

In [4]:
page = requests.get("https://raex-rr.com/ESG/ESG_companies/ESG_rating_companies/2025.1/").text
soup = BeautifulSoup(page, "html.parser")

In [5]:
table = soup.find("table", {"class": "rrp_table"})

In [6]:
head = [th.text for th in table.find("thead").find_all("th")]
rows = []
for row in table.find("tbody").find_all("tr"):
    # first two columns html type th and the rest is td
    parsed_row = [th.text.replace("\n", "").strip() for th in row.find_all("th")]
    parsed_row.extend([td.text.replace("\n", "").strip() for td in row.find_all("td")])
    rows.append(parsed_row)

In [7]:
df = pd.DataFrame(rows, columns=head)
df.head()

,№,Название,Страна,MOEX,Подотрасль,Е - рэнк,S - рэнк,G - рэнк
0,1,ФосАгро,Россия,PHOR,Химическая промышленность,3,1,4
1,2,Норильский никель,Россия,GMKN,Горнодобывающая промышленность и металлургия,1,4,1
2,3,Сбербанк,Россия,SBER,Банки,6,5,2
3,4,АЛРОСА,Россия,ALRS,Горнодобывающая промышленность и металлургия,11,2,16
4,5,Полюс,Россия,PLZL,Горнодобывающая промышленность и металлургия,2,13,12


In [8]:
rspp_reports = pd.read_csv("../parsing/rspp_reports.csv", index_col=0)
rspp_reports.head()

,компания,сектор,год,тип отчета,ссылка на отчет,id
0,ПАО АНК «Башнефть»,Нефтегазовая,2009,ОУР,/upload/uf/f99/ank_bashneft_oyp_2009.pdf,0
1,ПАО АНК «Башнефть»,Нефтегазовая,2010,ОУР,/upload/uf/b16/Doc1.pdf,1
2,ПАО АНК «Башнефть»,Нефтегазовая,2011,ОУР,/upload/uf/a26/ank_bashneft_oyp_2011.pdf,2
3,ПАО АНК «Башнефть»,Нефтегазовая,2012,ОУР,/upload/uf/6b4/ank_bashneft_oyp_2012.pdf,3
4,ПАО АНК «Башнефть»,Нефтегазовая,2013,ОУР,/upload/uf/b40/ank_bashneft_oyp_2013.pdf,4


In [9]:
df.to_csv("raex_esg.csv")

In [10]:
df["Название"].unique()

array(['ФосАгро', 'Норильский никель', 'Сбербанк', 'АЛРОСА', 'Полюс',
       'Московский кредитный банк', 'КазМунайГаз',
       'Корпоративный центр ИКС 5', 'Полиметалл', 'Роснефть',
       'Северсталь', 'En+ Group', 'Уралкалий', 'Группа «Татнефть»',
       'Московская Биржа', 'ОХК «Уралхим»', 'Росатом',
       'Магнитогорский металлургический комбинат (ММК)', 'НОВАТЭК',
       'ЭЛ5-Энерго', 'Русал', 'Т Плюс', 'ЕВРАЗ', 'Металлоинвест',
       'Казатомпром', 'Красцветмет', 'Казахтелеком', 'СИБУР Холдинг',
       'Аэрофлот – Российские авиалинии', 'ЛУКОЙЛ', 'Вымпелком',
       'Самрук-Энерго', 'НК «QazaqGaz»', 'КазТрансОйл', 'Пигмент (КРАТА)',
       'РЖД', 'НЛМК', 'Сетевая компания', 'Россети', 'Группа ЛСР', 'МТС',
       'Группа «Эйр Астана»', 'РусГидро', 'Отбасы банк',
       'Казахстанская компания по управлению электрическими сетями',
       'Сахалинская Энергия', 'Газпром', '«Магнезит», группа',
       'Росводоканал', 'Банк ВТБ', 'Юнипро', 'Селигдар',
       'Народный банк Казахста

In [11]:
companies = []
not_found = []
res = []
for idx, row in df.iterrows():
    c = row["Название"]
    year = row["Год последней оцененной отчетности"]
    row = pd.Series(row)

    find_left = -2
    find_right = -1
    if c.find("«") != -1:
        find_left = c.find("«") + 1
        find_right = c.find("»")
        c = c[find_left:find_right]
    if c.find('"') != -1:
        find_left = c.find('"') + 1
        find_right = c.find('"', find_left)
        c = c[find_left:find_right]
    if rspp_reports[rspp_reports["компания"].str.contains(c) & (rspp_reports["год"].str.contains(year))].shape[0] != 0:
        companies.append(c)
        rspp_c = rspp_reports[rspp_reports["компания"].str.contains(c) & (rspp_reports["год"].str.contains(year))]
        w = row
        q = rspp_c.to_dict()
        res.append(rspp_c.to_dict() | row.T.to_dict())
        # rssp_rows = pd.concat([rssp_rows, rspp_c]) if rssp_rows is not None else rspp_c
        # raex_rows = pd.concat([raex_rows, pd.DataFrame(]) if raex_rows is not None else pd.DataFrame(df.loc[idx]).T
    else:
        not_found.append(c)

KeyError: 'Год последней оцененной отчетности'

In [12]:
w, q

(№                                             140
 Название                              АБ «Россия»
 Код MOEX                                        -
 Подотрасль                                  Банки
 ESG-рейтинг                                    CC
 E Rank                                        149
 E-рейтинг                                       C
 S Rank                                        151
 S-рейтинг                                      CC
 G Rank                                        134
 G-рейтинг                                     CCC
 Год последней оцененной отчетности           2020
 E_transformed                                   9
 S_transformed                                   8
 G_transformed                                   7
 Name: 139, dtype: object,
 {'компания': {47: '«Coca-Cola HBC Россия»', 1376: 'ПАО «Энел Россия»'},
  'сектор': {47: 'Производство пищевых и других потребительских товаров',
   1376: 'Энергетика'},
  'год': {47: '2020', 1376: '2020'},


In [13]:
rspp_reports[rspp_reports["компания"].str.contains(c) & (rspp_reports["год"].str.contains(year))].shape[0] > 0

False

In [14]:
rspp_reports[rspp_reports["компания"].str.contains(c) & (rspp_reports["год"].str.contains(year))]

,компания,сектор,год,тип отчета,ссылка на отчет


In [15]:
c

'Нокиан Тайерс'

In [16]:
pd.DataFrame(res)

,компания,сектор,год,тип отчета,ссылка на отчет,№,Название,Код MOEX,Подотрасль,ESG-рейтинг,E Rank,E-рейтинг,S Rank,S-рейтинг,G Rank,G-рейтинг,Год последней оцененной отчетности,E_transformed,S_transformed,G_transformed
0,{714: 'ПАО «НЛМК»'},{714: 'Металлургическая и горнодобывающая'},{714: '2021'},{714: 'ИО'},{714: '/download/de45f8ddf53aa9ba80614d1c93e29...,1,НЛМК,NLMK,Чёрная металлургия,AA,2,AA,2,AA,21,A,2021,2,2,3
1,{887: 'ПАО «Полюс»'},{887: 'Металлургическая и горнодобывающая'},{887: '2021'},{887: 'ОУР'},{887: '/download/a79b0f48c2f54dc885ddf583ef4c4...,2,«Полюс»,PLZL,Драгоценные металлы,AA,1,AAA,14,A,27,A,2021,1,3,3
2,{1275: 'ПАО «Уралкалий»'},"{1275: 'Химическая, нефтехимическая, парфюмерн...",{1275: '2021'},{1275: 'ОУР'},{1275: '/download/0e88d52331c02d9939be212b2b95...,3,«Уралкалий»,-,Агрохимикаты,A,6,BBB,1,AA,6,AA,2021,4,2,2
3,{875: 'ПАО «Полиметалл»'},{875: 'Металлургическая и горнодобывающая'},{875: '2021'},{875: 'ИО'},{875: '/download/dd60bd2b772498057452e9b81e540...,5,«Полиметалл»,POLY,Драгоценные металлы,A,4,A,5,A,12,AA,2021,3,3,2
4,{1353: 'ПАО «ФосАгро»'},"{1353: 'Химическая, нефтехимическая, парфюмерн...",{1353: '2021'},{1353: 'ИО'},{1353: '/download/148f63c45d44e827ae38173e7add...,6,«ФосАгро»,PHOR,Агрохимикаты,A,8,BBB,11,A,5,AA,2021,4,3,2
5,{320: 'ПАО «ГМК «Норильский никель»'},{320: 'Металлургическая и горнодобывающая'},{320: '2021'},{320: 'ОУР'},{320: '/download/2540708cfd5daef1e7c1ed340f266...,8,"«Норильский никель», горно-металлургическая ко...",GMKN,Добыча прочих полезных ископаемых,A,12,BBB,18,A,4,AA,2021,4,3,2
6,{1121: 'ПАО «Северсталь»'},{1121: 'Металлургическая и горнодобывающая'},{1121: '2021'},{1121: 'ОУР'},{1121: '/download/4f81d2bbfa89029a7517772c9394...,9,«Северсталь»,CHMF,Чёрная металлургия,A,7,BBB,10,A,19,A,2021,4,3,3
7,{537: 'ПАО «ЛУКОЙЛ»'},{537: 'Нефтегазовая'},{537: '2021'},{537: 'ОУР'},{537: '/download/b1c943491484fd9e291de585e32d4...,10,«ЛУКОЙЛ»,LKOH,Интегрированные нефтегазовые компании,A,11,BBB,13,A,11,AA,2021,4,3,2
8,{637: 'ПАО «МТС»'},{637: 'Телекоммуникационная и связь'},{637: '2021'},{637: 'ОУР'},{637: '/download/0a53b7d4b6288f5721dc2ee12b3ba...,11,"ПАО ""МТС""",MTSS,Беспроводные телекоммуникационные услуги,A,35,B,4,A,3,AAA,2021,6,3,1
9,"{1142: 'БФ «Система»', 1143: 'АФК «Система»'}","{1142: 'Некоммерческие организации', 1143: 'Фи...","{1142: '2021', 1143: '2021'}","{1142: 'СО', 1143: 'ОУР'}",{1142: '/download/44cf6b555c9a653a9dba78b85d01...,12,АФК «Система»,AFKS,Финансовые услуги,A,14,BBB,34,BBB,1,AAA,2021,4,4,1


In [17]:
raex_rows.iloc[0].to_dict()

NameError: name 'raex_rows' is not defined

In [ ]:
not_found

['ЭЛ5-Энерго',
 'МОСКОВСКИЙ КРЕДИТНЫЙ БАНК',
 'Русал',
 'ЕВРАЗ',
 'Сахалинская энергия',
 'Магнитогорский металлургический комбинат (ММК)',
 'Интер РАО',
 'Вымпелком',
 'X5 Group',
 'En+ Group',
 'Аэрофлот - Российские авиалинии',
 'Сегежа Групп',
 'Металлоинвест',
 'СТЕПЬ',
 'ТМК (Трубная металлургическая компания)',
 'Селигдар',
 'ТрансКонтейнер',
 'Высочайший',
 'Акрон',
 'ФК Открытие',
 'Globaltrans',
 'Первая грузовая компания',
 'Кинросс Голд (Россия)',
 'М.Видео-Эльдорадо',
 'Объединенная металлургическая компания (ОМК)',
 'Проктер энд Гэмбл дистрибьюторская компания',
 'UPM Чудово',
 'Альфа-Банк',
 'Кузбассразрезуголь',
 'Юникредит Банк',
 'Совкомфлот',
 'Корпорация ВСМПО-АВИСМА',
 'Русагро',
 'Росбанк',
 'Global Ports',
 'Санкт-Петербург',
 'Тинькофф Банк',
 'Лента',
 'Райффайзенбанк',
 'Хенкель Рус',
 'ТНС ЭНЕРГО',
 'Архангельский ЦБК',
 'КуйбышевАзот',
 'Новороссийский морской торговый порт',
 'Россельхозбанк',
 'Метафракс кемикалс',
 'СИБУР Холдинг',
 'Транспортная группа F